## Card Game Suppress Seven

### Rules
    - Use the 52 cards in 4 colors: Heart, Diamond, Spade and Club.
    - Four players, each one gets 13 cards in the beginning.
    - The player who has Spade 7 on hand goes first.
    - The players must use the cards which have the same color and consecutive number to the cards on the board.
        - For example, if there is ♠7 on board, the next player must use ♠6, ♠8 or ♥7,♦7,♣7.
    - If a player has not card can be used, he/she needs to suppress one card. This card should not be seen by the other players until the end of this game.
    - The total points of the suppressed cards would be calculated at the end of the game, the one has the least points win.

In [14]:
class CustomError(Exception):
    """Base class for exceptions in this module."""
    pass

In [15]:
class CardNumberError(CustomError):
    def __init__(self, message, number):
        self.message = message
        self.number = number
    def __str__(self):
        return f"The numer {self.number} is {self.message} than the allowed card number range: 1~52."

In [17]:
# It is the card class, there are 52 cards in total, 13 cards for each suit. 
# So that we use the id between 1 and 52 to initialize the card.
class Card:
    suit_map = {0: 'Heart',
                1: 'Diamond',
                2: 'Spade',
                3: 'Club'}
    def __init__(self, number):
        if number > 52:
            raise CardNumberError('larger', number)
        elif number < 1:
            raise CardNumberError('smaller', number)
        else:
            self.number = (number-1)%13+1     # it is the number of this card, no matter which suit, 1-13
            self.suit = Card.suit_map[(number-1)//13] # it is the suit, one of the four
            self.card_id = number              # it is the id of the card, each card has an unique id, 1-52
            self.name = self.name()
    def __str__(self):
        return f"This card is {self.suit} {self.number}. The uid of this card is {self.card_id}."
    
    def name(self):
        return f"{self.suit} {self.number}"

In [18]:
# test
test_card = Card(60)

CardNumberError: The numer 60 is larger than the allowed card number range: 1~52.

In [8]:
test_card

In [10]:
test_card.name()

'Diamond 7'

In [12]:
print(test_card)

This card is Diamond 7. The uid of this card is 20.
